# LangChain + GitHub Models + Azure OpenAI

LangChain is a modular framework designed to streamline the development and integration of applications utilizing large language models (LLMs) by providing tools for effective prompt engineering, data handling, and complex workflows.

LangChain supports a number of model providers including Azure OpenAI and Cohere. Below find examples of how LangChain can be used with these models provided by the GitHub Models service.


## 1. Install dependencies

In [2]:
%pip install langchain-openai
%pip install langchain-community
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2. Constructing LangChain's ChatOpenAI class

Setting up the OpenAI API key and base URL for LangChain is the same as for OpenAI's Python client. You can set the `OPENAI_API_KEY` and `OPENAI_API_BASE` environment variables, or pass them directly to the `ChatOpenAI` class.


In [3]:
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

GPT_MODEL = "gpt-4o-mini"

llm = ChatOpenAI(model=GPT_MODEL)

In [4]:
from langchain_core.prompts import ChatPromptTemplate

# Create prompt 
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that loves cricket"),
    ("user", "{usertext}")
])

In [5]:
# Create chain with prompt and llm 
chain = prompt | llm

Now you can use the ChatOpenAI class to interact with the OpenAI API. 

In [7]:
chain.invoke({"usertext": f"Who won the 2024 cricket WC?"})

AIMessage(content="I'm sorry, but I don't have information on events that occurred after my last update in October 2021. For the latest updates on the 2024 Cricket World Cup, I recommend checking official cricket websites or news sources.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 28, 'total_tokens': 73}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_80a1bad4c7', 'finish_reason': 'stop', 'logprobs': None}, id='run-511bea75-8c8d-4678-bf78-e2b76b391ced-0', usage_metadata={'input_tokens': 28, 'output_tokens': 45, 'total_tokens': 73})

Looks like the model is not up to date with the latest contest. Let's help it out with some content from the wikipedia:

In [8]:
from langchain_community.document_loaders import WebBaseLoader

os.environ["USER_AGENT"] = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"

loader = WebBaseLoader("https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166/india-vs-south-africa-final-1415755/match-report")

docs = loader.load()
len(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


12648

In [30]:
docs[0].page_content


'T20 World Cup 2024, IND vs SA Final Match Report, June 29, 2024 - Bumrah and Hardik script stunning comeback to lead India to T20 World Cup gloryMatches (10)SCO vs AUS (1)RHF Trophy (4)T20 Blast (2)CPL 2024 (2)PAK vs BAN (1) Previous1st T20I\xa0•\xa0 EdinburghScotlandAustraliaToday1:00 PMMatch starts in  1  hr  47  minsScheduleVideosSeriesLive\xa0•\xa049th Match\xa0•\xa0RHF Trophy\xa0•\xa0 LeedsVP(28.4/50 ov) 115/2NDVipers chose to bat.ScheduleTableSeriesLive\xa0•\xa050th Match\xa0•\xa0RHF Trophy\xa0•\xa0 RadlettWS(23.5/50 ov) 84/6SUNRSunrisers chose to field.ScheduleTableSeriesLive\xa0•\xa051st Match\xa0•\xa0RHF Trophy\xa0•\xa0 SolihullSES(24.5/50 ov) 110/3CSSparks chose to field.ScheduleTableSeriesLive\xa0•\xa052nd Match\xa0•\xa0RHF Trophy\xa0•\xa0 SaleBLZ(25.5/50 ov) 108/3THDERThunder chose to field.ScheduleTableSeries2nd Quarter Final\xa0•\xa0T20 Blast\xa0•\xa0 T20\xa0•\xa0 HoveSussexLancashireToday6:00 PMMatch yet to beginScheduleTableVideosSeries7th Match\xa0•\xa0CPL 2024\xa0•\x

Due to the limits of the free service, the content retrieved from Wikipedia is too large to be added to the context -- that is ok since we can use a vector store to index the content. For that we need an embedding model which we can use with the OpenAIEmbeddings class. Note that we need to reduce the chunk size due to the token limits of the GitHub Models free service. 

In [9]:
from langchain_openai import OpenAIEmbeddings

EMBEDDINGS_MODEL = "text-embedding-3-small"

# need to constrain the chunk_size to work within the limits of the GitHub Model service
embeddings = OpenAIEmbeddings(model=EMBEDDINGS_MODEL,chunk_size=30)

Now, we will use the [FAISS](https://github.com/facebookresearch/faiss) library to index the vectors for a similarity search.

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Next, we can create a document chain that will use some context and the user's question to provide an answer:

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""You are a cricket geek who loves stats.
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We could just provide the context directly to the chain like so:

In [12]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Who won 2024 T20 World Cup Finals?",
    "context": [Document(page_content="India vs South Africa finals of T20 World Cup 2024")]
})

'The provided context does not specify the winner of the 2024 T20 World Cup Finals between India and South Africa.'

But we really want a retrieval chain that will retrieve the right documents from our vector index and then uses it as context in our prompt like so:

In [13]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Let's ask this one who won the 2024 ESC:

In [14]:
response = retrieval_chain.invoke({"input": f"Who won 2024 T20 World Cup Finals?"})
print(response["answer"])


India won the 2024 T20 World Cup Finals against South Africa by 7 runs.


And a few more increasingly detailed questions:

In [15]:
response = retrieval_chain.invoke({"input": "Who was the man of the match in the T20 World Cup Finals?"})
print(response["answer"])

The man of the match in the T20 World Cup Finals was Virat Kohli, who scored 76 runs off 59 balls.


In [16]:
response = retrieval_chain.invoke({"input": "which stadium the cricket final took place ?"})
print(response["answer"])

The cricket final took place at the Kensington Oval in Barbados.


In [17]:
response = retrieval_chain.invoke({"input": "Who had the best bowling performance in T20 World Cup finals?"})
print(response["answer"])

The best bowling performance in T20 World Cup finals, based on the provided context, was by Jasprit Bumrah, who took 2 wickets for 18 runs in his four overs.


In [18]:
response = retrieval_chain.invoke({"input": "What was the turning point in the T20 World Cup finals?"})
print(response["answer"])

The turning point in the T20 World Cup finals occurred in the 17th over when Hardik Pandya took the crucial wicket of Heinrich Klaasen, who was threatening to lead South Africa to victory. Klaasen had been in excellent form, having scored 52 runs, but his dismissal changed the momentum of the match. Following this, South Africa struggled to score runs, and India's bowlers, particularly Jasprit Bumrah and Arshdeep Singh, delivered tight overs that further put pressure on the South African batting lineup. This shift in the game's flow ultimately contributed to India's seven-run victory.


In [19]:
response = retrieval_chain.invoke({"input": "Organize results into a table with Date of the Match, Teams played, Man of the Match, Scores of both teams, Winning Team and Venue"})
print(response["answer"])

Here's the organized table based on the provided context:

| Date of the Match | Teams Played          | Man of the Match | Scores of Both Teams             | Winning Team | Venue        |
|-------------------|-----------------------|------------------|----------------------------------|--------------|--------------|
| June 29, 2024     | India vs South Africa  | Virat Kohli      | India: 176/7 <br> South Africa: 169/8 | India        | Barbados     |
